# New Discussion Tool Contributor Opt-Out Analysis

[Task](https://phabricator.wikimedia.org/T294491)

The New Discussion Tool is currently available as a opt-in beta feature at all Wikis. Prior to starting the A/B test of the New Discussion Tool,  we would like to know whether a significant number of people disabled the tool as a preference after using it to publish an edit.

**Requirements**: An analysis is done that enables us to see the percentage of contributors that explicitly disabled the New Discussion Tool after making at least one non-reverted edit with it, grouped by experience level and wiki.

**Relevant Dates**:
* 18 February 2021: Available at Arabic, Czech and Hungarian Wikipedias
* 11 March 20201: At More Wikipedia
* 16 March 2021: Available at English and Russian Wikipedias


## Data:

We reviewed the mediawiki [user_properties table](https://www.mediawiki.org/wiki/Manual:User_properties_table) to determine the current number of new discussion tool users that have the `discussiontools-betaenable` or `discussiontools-newtopictool` property currently disabled. 

Some notes regarding this dataset:
* This reflects all current nondefault user preferences.  User property records are added to the database when they differ from their default value.
* Data reflects the current state and does not account for users that have changed this preference multiple times.
* There are contributors that have used the new discussion tool but don't have a preference set in the user properties table. Possible reasons for this include: (1) the user  disabled the setting by selecting 'restore all default preferences' in their user preferences; (2) the user enabled discussion tools in their global preferences but not in their local preferences; or (3) The user disabled all discussion tool preferences by opting out of the discussion tool beta preference (applicable for the `Enable quick topic adding` preference only).

**Notes re how preference values are set**

* When you go into beta features and enable, the new topic preference is enabled by default: beta preference set to 1; No preference recorded for new topic option
* When you go into preferences and disable the new topic preference: beta preference set to 1: new topic pref set to 0
* When you go into proferences and renable the new topic preference: beta preference set to 1: No preference recorded for new topic option 
* When you go into into beta preferences and disable all discussiontool beta preference: beta preference set to 0; No preference recorded for new topic option

In [29]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

In [30]:
HIVE_SNAPSHOT = "2021-11"
START_OF_DATA = "2021-02-18"
END_OF_DATA = "2021-12-01"

## Collect new discussion tool contributors

In [49]:
#all users that made at least one non reverted (within 48 hours) edit with the new discussion tool since deployment

query = """

SELECT
    event_user_id as new_dt_user,
    wiki_db as wiki,
    CASE
        WHEN min(event_user_revision_count) < 100 THEN 'under 100'
        WHEN (min(event_user_revision_count) >=100 AND min(event_user_revision_count <= 500)) THEN '100-500'
        ELSE 'over 500'
        END AS edit_count_group
FROM wmf.mediawiki_history AS mh
WHERE 
    ARRAY_CONTAINS(revision_tags, 'discussiontools-newtopic') 
    AND snapshot = '2021-11' 
-- date first deployed
    AND event_timestamp >= '2021-02-18'  
    AND event_timestamp < '2021-12-01'   
-- only on desktop
    AND NOT array_contains(revision_tags, 'iOS')
    AND NOT array_contains(revision_tags, 'Android')
    AND NOT array_contains(revision_tags, 'Mobile Web')
     -- find all edits on talk pages 
    AND page_namespace_historical % 2 = 1
    AND event_entity = 'revision' AND 
    event_type = 'create'
    AND event_user_is_anonymous = FALSE
    -- remove all bots
    AND SIZE(event_user_is_bot_by_historical) = 0  -- not a bot
    AND NOT (revision_is_identity_reverted AND 
            revision_seconds_to_identity_revert <= 172800) -- not reverted within 48 hours
GROUP BY
    event_user_id,
    wiki_db
"""

In [50]:
new_dt_user = hive.run(
    query.format(
        hive_snapshot = HIVE_SNAPSHOT,
        START_OF_DATA= START_OF_DATA,
        END_OF_DATA=END_OF_DATA
    )
)

In [51]:
#Total new dt users

Total_dt_users = new_dt_user['new_dt_user'].count()

print('Total number of new discussion users for whom we will be checking beta preferences:' , Total_dt_users)

Total number of new discussion users for whom we will be checking beta preferences: 9880


## New Discussion Tool Users Current Preference Status

The are two possible ways to disable the new discussion tool:
(1) By explicilty disabling the `Enable quick topic adding` option in your preferences OR
(2) By explcility disabling the discussion tool beta preference.

We'll review the opt out rate by both of these methods.

### Percentage of Users that explicilty disable the `Enable quick topic adding` option 

In [64]:
#Querying user_properties for getting the new topic tools preference set by the new dt contributors we got in the above query

query='''
SELECT 
  user_properties.up_value AS preference,
  user_properties.up_user AS `user`
FROM user_properties 
WHERE user_properties.up_user in ({users})
AND up_property = "discussiontools-newtopictool"
'''

In [65]:
# Looping through each wiki for the list of users for each skin

wikis = new_dt_user['wiki'].unique()
up_pref=list()
for wiki in wikis:
    user_ids = new_dt_user[new_dt_user['wiki'] == wiki]["new_dt_user"]
    user_list = ','.join([str(u) for u in user_ids])
    prefs = mariadb.run(
      query.format(users=user_list),
      wiki
    )
    up_pref.append(prefs)

pref= pd.concat(up_pref)

In [66]:
# Join with edit count data from mediawiki_history
new_dt_user_pref = new_dt_user.join(pref.set_index('user'), on = 'new_dt_user', how = 'left')

In [67]:
# covert skin column to string type 
new_dt_user_pref['preference'] = new_dt_user_pref['preference'].astype(str)

In [68]:
# rename values. 
pref_aliases = {
    "b\'0\'":"explicitly disabled",
    "nan": "enabled or no local preference recorded"
}

new_dt_user_pref= new_dt_user_pref.replace({"preference": pref_aliases})

#### Overall Opt-In Rate

In [69]:
new_dt_pref_overall= new_dt_user_pref[['preference', 'new_dt_user']].groupby('preference').count()

new_dt_pref_overall

,new_dt_user
preference,
enabled or no local preference recorded,9819
explicitly disabled,61


In [70]:
pct_user_opt_rate =(100. * new_dt_pref_overall / new_dt_pref_overall.sum()).round(2).astype(str) + '%'
pct_user_opt_rate.sort_values(by=['new_dt_user'],ascending=False)

,new_dt_user
preference,
enabled or no local preference recorded,99.38%
explicitly disabled,0.62%


Overall, only 0.6% of all new discussion tool users currently have the `discussiontool-newtopictool` preference explicitly set as disabled in the User Properties table. 

#### By Experience Level

In [71]:
# calculate total enabled and disabled
new_dt_pref_byexp = new_dt_user_pref[['preference', 'edit_count_group' ,'new_dt_user']].groupby(['edit_count_group','preference']).count()

new_dt_pref_byexp

new_dt_user
edit_count_group preference                                          
100-500          enabled or no local preference recorded         1300
                 explicitly disabled                                2
over 500         enabled or no local preference recorded         5676
                 explicitly disabled                               47
under 100        enabled or no local preference recorded         2843
                 explicitly disabled                               12

In [72]:
pct_user_opt_rate_byexp =(100. * new_dt_pref_byexp/ new_dt_pref_byexp.groupby('edit_count_group').sum()).round(2).astype(str) + '%'
pct_user_opt_rate_byexp

new_dt_user
edit_count_group preference                                         
100-500          enabled or no local preference recorded      99.85%
                 explicitly disabled                           0.15%
over 500         enabled or no local preference recorded      99.18%
                 explicitly disabled                           0.82%
under 100        enabled or no local preference recorded      99.58%
                 explicitly disabled                           0.42%

#### Wikis with highest opt out rate

In [155]:
# calculate total enabled and disabled
new_dt_user_pref_bywiki = new_dt_user_pref[['preference', 'wiki' ,'new_dt_user']].groupby(['wiki','preference']).count()

In [156]:
pct_user_opt_rate_bywiki =(100. * new_dt_user_pref_bywiki/ new_dt_user_pref_bywiki.groupby('wiki').sum()).round(2)

In [157]:
# View Wikis with Highest % of users that explciitly disabled the feature
top_wikis = pct_user_opt_rate_bywiki.filter(like = "disabled", axis = 0).sort_values(by=['new_dt_user'], ascending = False)
top_wikis.head()

,,new_dt_user
wiki,preference,
cowiki,explicitly disabled,100.00
test2wiki,explicitly disabled,100.00
viwikivoyage,explicitly disabled,33.33
glwiki,explicitly disabled,18.18
viwikibooks,explicitly disabled,12.50


#### English and Spanish Wikipedias

In [73]:
# filter to target wikis
target_wiki = ['enwiki', 'eswiki']
new_dt_user_pref_wiki= new_dt_user_pref[new_dt_user_pref['wiki'].isin(target_wiki)]


In [74]:
# calculate total enabled and disabled
new_dt_pref_wiki = new_dt_user_pref_wiki[['preference', 'wiki' ,'new_dt_user']].groupby(['wiki','preference']).count()

new_dt_pref_wiki

new_dt_user
wiki   preference                                          
enwiki enabled or no local preference recorded         2165
       explicitly disabled                               14
eswiki enabled or no local preference recorded          610
       explicitly disabled                                3

In [75]:
pct_user_opt_rate_wiki =(100. * new_dt_pref_wiki/ new_dt_pref_wiki.groupby('wiki').sum()).round(2).astype(str) + '%'
pct_user_opt_rate_wiki

new_dt_user
wiki   preference                                         
enwiki enabled or no local preference recorded      99.36%
       explicitly disabled                           0.64%
eswiki enabled or no local preference recorded      99.51%
       explicitly disabled                           0.49%

### Percentage of Users that explicilty disable the Discussion Tools Beta Preference

In [91]:
#Querying user_properties to obtain new dt contributors that disabled the Discussion Tools Beta Preference

query='''
SELECT 
  user_properties.up_value AS preference,
  user_properties.up_user AS `user`
FROM user_properties 
WHERE user_properties.up_user in ({users})
AND up_property = "discussiontools-betaenable"
'''              

In [92]:
# Looping through each wiki for the list of users for each skin

wikis = new_dt_user['wiki'].unique()
up_pref=list()
for wiki in wikis:
    user_ids = new_dt_user[new_dt_user['wiki'] == wiki]["new_dt_user"]
    user_list = ','.join([str(u) for u in user_ids])
    prefs = mariadb.run(
      query.format(users=user_list),
      wiki
    )
    up_pref.append(prefs)

pref= pd.concat(up_pref)

In [93]:
# Join with edit count data from mediawiki_history
new_dt_beta_pref = new_dt_user.join(pref.set_index('user'), on = 'new_dt_user', how = 'left')

In [94]:
new_dt_beta_pref.head()

,new_dt_user,wiki,edit_count_group,preference
0,15871,nlwiki,over 500,b'1'
1,59964,zhwiki,over 500,b'0'
2,439868,itwiki,over 500,NaN
3,558149,cswiki,under 100,NaN
4,721473,viwiki,over 500,b'1'


In [96]:
# covert skin column to string type 
new_dt_beta_pref['preference'] = new_dt_beta_pref['preference'].astype(str)

In [97]:
# rename values. 
pref_aliases = {
    "b\'0\'":"explicitly disabled beta preference",
    "b\'1\'":"explicitly enabled beta preference",
    "nan": "no local preference recorded"
}

new_dt_beta_pref= new_dt_beta_pref.replace({"preference": pref_aliases})

#### Overall Opt In Rate

In [98]:
new_dt_beta_pref_overall= new_dt_beta_pref[['preference', 'new_dt_user']].groupby('preference').count()

new_dt_beta_pref_overall

,new_dt_user
preference,
explicitly disabled beta preference,742
explicitly enabled beta preference,6301
no local preference recorded,2883


In [99]:
pct_beta_opt_rate =(100. * new_dt_beta_pref_overall / new_dt_beta_pref_overall.sum()).round(2).astype(str) + '%'
pct_beta_opt_rate.sort_values(by=['new_dt_user'],ascending=False)

,new_dt_user
preference,
explicitly disabled beta preference,7.48%
explicitly enabled beta preference,63.48%
no local preference recorded,29.04%


#### By Experience Level

In [83]:
# calculate total enabled and disabled
new_dt_beta_pref_byexp = new_dt_beta_pref[['preference', 'edit_count_group' ,'new_dt_user']].groupby(['edit_count_group','preference']).count()

new_dt_beta_pref_byexp

new_dt_user
edit_count_group preference                                      
100-500          explicitly disabled beta preference           61
                 explicitly enabled beta preference           887
                 no local preference recorded                 358
over 500         explicitly disabled beta preference          529
                 explicitly enabled beta preference          3770
                 no local preference recorded                1461
under 100        explicitly disabled beta preference          151
                 explicitly enabled beta preference          1644
                 no local preference recorded                1065

In [84]:
pct_beta_opt_rate_byexp =(100. * new_dt_beta_pref_byexp/ new_dt_beta_pref_byexp.groupby('edit_count_group').sum()).round(2).astype(str) + '%'
pct_beta_opt_rate_byexp

new_dt_user
edit_count_group preference                                     
100-500          explicitly disabled beta preference       4.67%
                 explicitly enabled beta preference       67.92%
                 no local preference recorded             27.41%
over 500         explicitly disabled beta preference       9.18%
                 explicitly enabled beta preference       65.45%
                 no local preference recorded             25.36%
under 100        explicitly disabled beta preference       5.28%
                 explicitly enabled beta preference       57.48%
                 no local preference recorded             37.24%

# Wikis with the highest opt-out rate

In [158]:
# calculate total enabled and disabled
new_dt_beta_pref_bywiki = new_dt_beta_pref[['preference', 'wiki' ,'new_dt_user']].groupby(['wiki','preference']).count()

In [159]:
pct_beta_opt_rate_bywiki =(100. * new_dt_beta_pref_bywiki/ new_dt_beta_pref_bywiki.groupby('wiki').sum()).round(2)


In [161]:
# View Wikis with Highest % of users that explciitly disabled the feature
top_wikis = pct_beta_opt_rate_bywiki.filter(like = "disabled", axis = 0).sort_values(by=['new_dt_user'], ascending = False)
top_wikis.head(10)

,,new_dt_user
wiki,preference,
wuuwiki,explicitly disabled beta preference,100.00
vecwiki,explicitly disabled beta preference,100.00
knwiki,explicitly disabled beta preference,50.00
ruwikibooks,explicitly disabled beta preference,50.00
ruwikisource,explicitly disabled beta preference,50.00
siwiki,explicitly disabled beta preference,50.00
urwiki,explicitly disabled beta preference,50.00
jawikibooks,explicitly disabled beta preference,33.33
cswiki,explicitly disabled beta preference,29.15
